## Trabalhando com dados complexos
Neste notebook veremos como trabalhar com colunas que armazenam listas e dicionários <br>
Utilizaremos os comandos  struct, split, size, array_contains, explode, create_map

### Importando as bibliotecas
Nesta etapa iremos apenas importar todas as bibliotecas e funções necessárias para rodar o programa

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, struct, split, size, array_contains, explode, create_map
from pyspark.sql.types import StructField, StructType, StringType, LongType, DoubleType, IntegerType

### Criando uma SparkSession
Por meio de uma SparkSession terei acesso ao SparkContext da minha aplicação.

In [5]:
# Inicia uma sparkSession
spark = SparkSession \
    .builder \
    .appName("Meu curso de pyspark") \
    .getOrCreate()

### Criando um Dataframe manualmente 
Iremos criar um Dataframe manualmente para este notebook

In [7]:
# Carrega dados usando schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("job", StringType(), True)
])

# Cria as linhas do nosso futuro dataframe
newRows = [
    [30, "Cientista de dados"],
    [20, "Dev Java"],
    [10, None]
]

# Cria um RDD de Rows
parallelizedRows = spark.sparkContext.parallelize(newRows)

# Cria um dataframe a partir do RDD que criamos anteriormente
dados_manual = spark.createDataFrame(parallelizedRows, schema)

# Mostra as informações do dataframe
dados_manual.show()

+---+------------------+
 id| job|
+---+------------------+
 30|Cientista de dados|
 20| Dev Java|
 10| null|
+---+------------------+

### Trabalhando com colunas que armazenam listas

**Colunas do tipo struct -> Colocando id e job numa unica coluna do tipo struct**

In [10]:
dados_complexos = dados_manual.select(struct("id", "job").alias("complexo"))
dados_complexos.select("complexo.job").show(5)

+------------------+
 job|
+------------------+
Cientista de dados|
 Dev Java|
 null|
+------------------+

**Função Split -> Separa as informações da coluna job quando encontrar espaço. O resultado será um array**

In [12]:
dados_split = dados_manual.select(split(col("job"), " ").alias("split_column"))
dados_split.show()

+--------------------+
 split_column|
+--------------------+
[Cientista, de, d...|
 [Dev, Java]|
 null|
+--------------------+

**Podemos referenciar nosso array pela sua posição**

In [14]:
dados_split.selectExpr("split_column[0]").show()

+---------------+
split_column[0]|
+---------------+
 Cientista|
 Dev|
 null|
+---------------+

**Tammém podemos contar quantas posições temos em cada array**

In [16]:
dados_split.select(size(col("split_column"))).show()

+------------------+
size(split_column)|
+------------------+
 3|
 2|
 -1|
+------------------+

**A função array_contains nos permite verificar se um determinado elemento existe no array**

In [18]:
dados_split.select(array_contains(col("split_column"), "Dev")).show()

+---------------------------------+
array_contains(split_column, Dev)|
+---------------------------------+
 false|
 true|
 null|
+---------------------------------+

**Por fim, a função explore pode ser aplicada em colunas Array para extrair cada elemento em uma nova linha**

In [20]:
dados_split.select(explode(col("split_column"))).show()

+---------+
 col|
+---------+
Cientista|
 de|
 dados|
 Dev|
 Java|
+---------+

### Trabalhando com colunas que armazenam dicionários

**Se você quiser trabalhar com conjunto de valores do tipo "key-value", pode trabalhar com o tipo complexo MAP**

In [23]:
dados_map = dados_manual.select(create_map(col("id"), col("job")).alias("mapa"))
dados_map.show()

+--------------------+
 mapa|
+--------------------+
[30 -> Cientista ...|
 [20 -> Dev Java]|
 [10 ->]|
+--------------------+

**Depois podemos usar a coluna map da seguinte forma**

In [25]:
dados_map.selectExpr("mapa['30']").show()

+---------------------+
mapa[CAST(30 AS INT)]|
+---------------------+
 Cientista de dados|
 null|
 null|
+---------------------+

**Também é possível aplicar a função explode em uma columa map para retornar as colunas key -> value**

In [27]:
dados_map.selectExpr("explode(mapa)").show()

+---+------------------+
key| value|
+---+------------------+
 30|Cientista de dados|
 20| Dev Java|
 10| null|
+---+------------------+

### Obrigado!
Quer construir uma carreira em Data Science? Acesse meu blog pessoal em https://www.hackinganalytics.com/